# 机器学习解决中文文本分类问题

![jupyter](./imgs/chinese_nlp.jpg)

### 词袋模型/TF-IDF + 分类模型(LR\SVM\XGBoost等)
### 中文文本分类传统流程：分词 -> 去停用词 -> TF-IDF特征抽取 -> 分类模型

## 1.中文分词
### https://github.com/fxsjy/jieba

In [1]:
import warnings
warnings.filterwarnings('ignore')

import jieba

seg_list = jieba.cut("武汉加油！中国加油！没有一个冬天不可逾越，没有一个春天不会来临。", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.738 seconds.
Prefix dict has been built successfully.


Default Mode: 武汉/ 加油/ ！/ 中国/ 加油/ ！/ 没有/ 一个/ 冬天/ 不可逾越/ ，/ 没有/ 一个/ 春天/ 不会/ 来临/ 。


## 2.词频向量化
CountVectorizer 类会将文本中的词语转换为词频矩阵  
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['疫情结束，一起去武汉看樱花！',
          '可怕！新冠疫情从武汉蔓延！',
          '军运会在武汉举行。',
          '武汉加油！',
        ]

# 精准模式进行分词
tokenzied_corpus = [' '.join(jieba.cut(sentence, cut_all=False)) for sentence in corpus]
tokenzied_corpus

['疫情 结束 ， 一起 去 武汉 看 樱花 ！', '可怕 ！ 新冠 疫情 从 武汉 蔓延 ！', '军运会 在 武汉 举行 。', '武汉 加油 ！']

In [3]:
# 定义词频矩阵提取器
vectorizer = CountVectorizer(min_df=1) 
# min_df: df为document frequence即词出现在所有文档中的次数，
# 最小文档频率意思为构造词频矩阵时选取的词汇要最少要出现在几篇文档内

X = vectorizer.fit_transform(tokenzied_corpus)  # 将词频矩阵提取器应用在分词后的语料
feature_name = vectorizer.get_feature_names()
print([(word, index) for (index, word) in enumerate(feature_name)])
print()
print("（文档索引，词汇索引）文档中的词频")
print(X)

[('一起', 0), ('举行', 1), ('军运会', 2), ('加油', 3), ('可怕', 4), ('新冠', 5), ('樱花', 6), ('武汉', 7), ('疫情', 8), ('结束', 9), ('蔓延', 10)]

（文档索引，词汇索引）文档中的词频
  (0, 8)	1
  (0, 9)	1
  (0, 0)	1
  (0, 7)	1
  (0, 6)	1
  (1, 8)	1
  (1, 7)	1
  (1, 4)	1
  (1, 5)	1
  (1, 10)	1
  (2, 7)	1
  (2, 2)	1
  (2, 1)	1
  (3, 7)	1
  (3, 3)	1


In [4]:
#将词频稀疏矩阵转化为一般矩阵
for w in X.toarray():
    print('\t'.join([str(round(x, 2)) for x in w]))

1	0	0	0	0	0	1	1	1	1	0
0	0	0	0	1	1	0	1	1	0	1
0	1	1	0	0	0	0	1	0	0	0
0	0	0	1	0	0	0	1	0	0	0


## 3.TF-IDF特征

有些词在文本中尽管词频高，但是并不重要，这个时候就可以用TF-IDF技术。  
某一特定文件内的高词语频率，以及该词语在整个文件集合中的低文件频率，可以产生出高权重的TF-IDF。   
因此，TF-IDF倾向于过滤掉常见的词语，保留重要的词语。 

这里使用TfidfTransformer将前面的词频矩阵转化为TF-IDF表示

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer

In [5]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer 

# TfidfTransformer在CountVectorizer提取的词频矩阵基础上构造，所以这里定义一个pipeline，将过程串起来
pipeline = Pipeline([('count', CountVectorizer()),
                 ('tfid', TfidfTransformer())])

# 训练pipeline
pipeline_model = pipeline.fit(tokenzied_corpus)

# 查看pipeline model中词频矩阵中的词汇，即TF中的term
print(pipeline_model['count'].get_feature_names())

['一起', '举行', '军运会', '加油', '可怕', '新冠', '樱花', '武汉', '疫情', '结束', '蔓延']


In [6]:
# 查看pipeline model中IDF,即逆词频

idfs = pipeline_model['tfid'].idf_
print('\t'.join([str(round(x, 2)) for x in idfs]))

1.92	1.92	1.92	1.92	1.92	1.92	1.92	1.0	1.51	1.92	1.92


In [7]:
# 对文本进行tfidf特征抽取
featrure = pipeline_model.transform(tokenzied_corpus)
# 查看tfidf矩阵
# weight就是模型的输入，它的大小(m,n),m是文本个数，n是词库的大小
weight = featrure.toarray()  # 将稀疏矩阵转化为一般矩阵，方便查看
for w in weight:
    print('\t'.join([str(round(x, 2)) for x in w]))

0.51	0.0	0.0	0.0	0.0	0.0	0.51	0.26	0.4	0.51	0.0
0.0	0.0	0.0	0.0	0.51	0.51	0.0	0.26	0.4	0.0	0.51
0.0	0.66	0.66	0.0	0.0	0.0	0.0	0.35	0.0	0.0	0.0
0.0	0.0	0.0	0.89	0.0	0.0	0.0	0.46	0.0	0.0	0.0


## 4. XGBoost模型

说到XGBoost，不得不提GBDT(Gradient Boosting Decision Tree)。因为XGBoost本质上还是一个GBDT，但是力争把速度和效率发挥到极致，所以叫X (Extreme) GBoosted。高效地实现了GBDT算法并进行了算法和工程上的许多改进。
- 对于非深度学习类型的机器学习分类问题，XGBoost是最流行的库。
- 由于XGBoost可以很好地扩展到大型数据集中，并支持多种语言，它在商业化环境中特别有用。
- XGBoost不仅学习效果很好，而且速度也很快，相比梯度提升算法在另一个常用机器学习库scikit-learn中的实现，XGBoost的性能经常有十倍以上的提升。

https://xgboost.readthedocs.io/en/latest/index.html

In [8]:
import re
import joblib
import numpy as np
import pandas as pd
import xgboost as xgb

### 4.1 数据处理：分词、去停用词

In [9]:
# 加载停用词典，可以自定义
with open('./data/baidu_stopwords.txt', 'r') as f:
    stop_words = f.read().split('\n')

stop_words[:10]

['--', '?', '“', '”', '》', '－－', 'able', 'about', 'above', 'according']

In [10]:
# 分词 -> 去停用词 -> 清除无用词

def cut_and_clean(text, stop_words):
    cuted_text = ' '.join([x for x in jieba.lcut(text) if x not in stop_words])
    clean_text = re.sub('([\.!\/_,?=\$%\^\)*\(\+\"\'\+——！:：；，。？、~@#%……&*（）·¥\-\|\\《》〈〉～《 》「」『』\{\}\|])', ' ', cuted_text)
    clean_text = re.sub('\s{2,}', ' ', clean_text)
    clean_text = clean_text.lower()
    return clean_text.strip()

text = '武汉的樱花很漂亮！'
cut_and_clean(text, stop_words)

'武汉 樱花 很漂亮'

In [11]:
# 对训练集数据进行处理

train_df = pd.read_csv('./processed_data/train.csv', encoding='utf-8')
train_df['processed_text'] = train_df['text'].apply(lambda text: cut_and_clean(text, stop_words))
train_df.head()

,text,label,processed_text
0,肺炎疫情,1,肺炎 疫情
1,【西方记者挑事提问，WHO总干事用事实力挺中国】12日，世界卫生组织大会上，有西方记者挑事提...,1,【 西方 记者 挑事 提问 who 总干事 用事 实力 挺 中国 】 12 日 世界卫生组织...
2,人数终于 始呈下降趋势。这个春节有喜有忧，但事情终有完结的那一天。中午出去买了一次蔬菜，过了...,1,人数 终于 始 呈 下降 趋势 春节 有喜有忧 事情 终 完结 中午 买 蔬菜 六道 关卡 ...
3,【抗病毒小刘上线】今天有好几个朋友来问我戴没戴口罩要不要寄口罩给我，超级感动，小刘除了被吓到...,1,【 抗病毒 小 刘 上线 】 好几个 朋友 问 戴 没 戴 口罩 寄 口罩 超级 感动 小 ...
4,潜伏期传染，后期会不会更惊讶O最新疫情地图出炉，这传播速度有让你觉得惊讶吗,1,潜伏期 传染 后期 会 更 惊讶 o 最新 疫情 地图 出炉 传播速度 惊讶


In [12]:
# 对测试集数据进行处理

test_df = pd.read_csv('./processed_data/test.csv', encoding='utf-8')
test_df['processed_text'] = test_df['text'].apply(lambda text: cut_and_clean(text, stop_words))
test_df.head()

,text,label,processed_text
0,星火线上课 足不出户，同样进步！L星火教育呼市的 视频,2,星火 线 上课 足不出户 l 星火 教育 呼市 视频
1,中科院病毒研究所石正丽老师演讲《追踪SARS的源头》探索病毒与人类社会的关系。真是好看,2,中科院 病毒 研究所 石正丽 老师 演讲 追踪 sars 源头 探索 病毒 人类 社会 关系 好看
2,嗯 @熊中默就是那个给那位挂基层辅警照片 暴平安天门几万条艾滋病巨婴gay洗地还挂plmm照...,0,熊中默 那位 挂 基层 辅警 照片 暴 平安 天门 几万 条 艾滋病 巨婴 gay 洗地 还...
3,今天看到两条让我真的笑，笑出声的 。「为防病毒，出门抢购双黄连，染上病毒。」O 「新冠病毒感...,0,两条 真的 笑 笑 出声 防病毒 出门 抢购 双黄连 染上 病毒 o 新冠 病毒感染 肾脏 ...
4,@_莫西干_,1,莫西 干


In [13]:
# 提取文本与标签

train_data, train_label = train_df['processed_text'], train_df['label']

test_data, test_label = test_df['processed_text'], test_df['label']

### 4. 2 文本特征化：词频 -> TF-IDF -> xgb Matrix

In [14]:
# 词频矩阵提取器
vectorizer = CountVectorizer(max_df=1000,  # 最大文档频率：词汇最多出现在多少篇文档中
                             min_df=50)    # 最小文档频率：词汇最少出现在多少篇文档中
# Tfidf提取器
tfidftransformer = TfidfTransformer()

# 定义特征提取pipeline
pipe = Pipeline([('count', vectorizer),
                 ('tfid', tfidftransformer)])

# 在训练集上进行pipeline构造
pipe_model = pipe.fit(train_data)

# 查看tfidf特征向量维度
print(f"Tfidf 特征向量维度为: {len(pipe_model['tfid'].idf_)}")

Tfidf 特征向量维度为: 4013


In [15]:
# 保存pipeline model
pipe_save_path = './xgb/pipe.pkl'
joblib.dump(pipe_model, pipe_save_path)

['./xgb/pipe.pkl']

In [16]:
# 对训练与测试集文本进行tfidf特征抽取
train_preds = pipe_model.transform(train_data)
train_array = train_preds.toarray()

test_preds = pipe_model.transform(test_data)
test_array = test_preds.toarray()

In [17]:
# 查看训练集特征向量形状
train_array.shape

(79816, 4013)

#### XGBoost数据集

在XGBoost中数据的加载是使用其特有的数据格式进行训练的，即xgb.DMatrix。   
之前说过样本分布不均衡问题，这里我们在构建数据集时通过对样本权重的加权，提高样本少类别的权重。   
xgb.DMatrix设置每一个样本的权重，这样模型在计算损失的过程中都会结合每个样本的权重去计算。   
这里我们提高类别较少的0，2类别的权重。  
##### Loss = 0.2 * Loss0 + 0.1 * Loss1+ 0.2 * Loss2   

In [18]:
label_weight_dict = {
    0: 0.2,
    1: 0.1,
    2: 0.2
}

sample_weight = [label_weight_dict[label] for label in train_label]
sample_weight[:10]

[0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.2, 0.2, 0.1]

In [19]:
# 将训练与测试数据转化为xgboost的DMatrix形式，并对样本重要性进行加权
dtrain = xgb.DMatrix(train_array, label=train_label, weight=sample_weight)
dtest = xgb.DMatrix(test_array)

### 4.3 模型训练

XGBoost参数调优完全指南：https://blog.csdn.net/han_xiaoyang/article/details/52665396   
XGBoost 重要参数(调参使用): https://www.cnblogs.com/TimVerion/p/11436001.html   

XGBoost的训练API：https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training   
包括： 
- xgboost.train(params, dtrain, num_boost_round=10, evals=(), obj=None, feval=None, maximize=None, early_stopping_rounds=None, evals_result=None, verbose_eval=True, xgb_model=None, callbacks=None)¶    

- xgboost.cv(params, dtrain, num_boost_round=10, nfold=3, stratified=False, folds=None, metrics=(), obj=None, feval=None, maximize=None, early_stopping_rounds=None, fpreproc=None, as_pandas=True, verbose_eval=None, show_stdv=True, seed=0, callbacks=None, shuffle=True)

可以先通过xgboost.cv进行交叉验证选出最合适的num_boost_round，在通过xgboost.train进行训练

In [20]:
# param = {'max_depth':6, 
#          'eta':0.1, 
#          'eval_metric':'merror', 
#          'objective':'multi:softmax', 
#          'num_class': 3}

# max_n_estimators = 1000

# cv_res= xgb.cv(param,
#                dtrain,
#                num_boost_round=max_n_estimators,  # 最大迭代次数
#                early_stopping_rounds=20, # 没有提升迭代停止，输出最好的轮数
#                nfold=5,  # n折交叉
#                metrics='auc',  # 评价指标
#                show_stdv=True,  # 打印交叉验证的标准差
#                verbose_eval=10 #每10轮打印指标
#                )

# cv_res

# #cv_res.shape[0]为最佳迭代次数
# best_num_boost_round = cv_res.shape[0] 

# bst = xgb.train(param,
#                 dtrain,
#                 num_boost_round=best_num_boost_round)

In [21]:
param = {'max_depth': 6,  # 树的最大深度，一般3-10
         'eta':0.1,  # learning rate每一步迭代的步长，很重要。太大了运行准确率不高，太小了运行速度慢，一般0.01-0.2
         'eval_metric':'merror', # 评估度量方法，merror对应多分类错误率
         'objective':'multi:softmax', # 损失函数，这里设置为使用softmax的多分类器，下面还需设置num_class
         'num_class': 3}

best_num_boost_round = 100 # 生成的最大树的数目，也是最大的迭代次数, 这里通过cross validation得到最优的num_boost_round

# XGBoost模型训练
bst = xgb.train(param,
                dtrain,
                num_boost_round=best_num_boost_round)

In [22]:
# 保存训练好的模型
model_save_path = './xgb/xgb.json'
bst.save_model(model_save_path)

## 5. 模型评估

In [23]:
from sklearn.metrics import classification_report

In [24]:
# 模型预测
preds = bst.predict(dtest)

In [25]:
# 评估分类结果
result = classification_report(test_label, preds)
print(result)

              precision    recall  f1-score   support

           0       0.55      0.30      0.39      1796
           1       0.66      0.82      0.73      5653
           2       0.61      0.47      0.53      2551

    accuracy                           0.64     10000
   macro avg       0.61      0.53      0.55     10000
weighted avg       0.63      0.64      0.62     10000



## 6. 模型预测
### 数据预处理-> 分词 -> 去停用词 -> TF-IDF特征抽取 -> xgb模型预测

In [26]:
# 加载停用词
with open('./data/baidu_stopwords.txt', 'r') as f:
    stop_words = f.read().split('\n')

#加载特征抽取模型
pipe_save_path = './xgb/pipe.pkl'
pipe_model = joblib.load(pipe_save_path)

#加载机器学习模型
model_save_path = './xgb/xgb.json'
xgb_model = xgb.Booster()
xgb_model.load_model(model_save_path)

In [27]:
predict_sentences = [
    "因为疫情被困家里2个月了，好压抑啊，感觉自己抑郁了！",
    "我国又一个新冠病毒疫苗获批紧急使用。",
    "我们在一起，打赢这场仗，抗击新馆疫情，我们在行动！"]

# 进行分词与预处理
predict_tokenizes = [cut_and_clean(sentence, stop_words) for sentence in predict_sentences]

predict_tokenizes

['疫情 被困 家里 2 月 好 压抑 感觉 抑郁', '我国 一个 新冠 病毒 疫苗 获批 紧急', '打赢 这场 仗 抗击 新馆 疫情']

In [28]:
# 提取TF-IDF特征
features_array = pipe_model.transform(predict_tokenizes).toarray()
features_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
# 转化为xgb的DMatrix
dpredict = xgb.DMatrix(features_array)

# 使用模型进行预测
predict_result = xgb_model.predict(dpredict)

# 将标签还原为[-1, 0, 1]
predict_result = [int(label - 1) for label in predict_result]
predict_result

[0, 0, 1]

In [30]:
# 展示结果
for text, label in zip(predict_sentences, predict_result):
    print(f'Text: {text}\nPredict: {label}')

Text: 因为疫情被困家里2个月了，好压抑啊，感觉自己抑郁了！
Predict: 0
Text: 我国又一个新冠病毒疫苗获批紧急使用。
Predict: 0
Text: 我们在一起，打赢这场仗，抗击新馆疫情，我们在行动！
Predict: 1
